<a href="https://colab.research.google.com/github/JinHuiXu1991/Jin_DATA606/blob/main/ipynb/DATA606_Part2_CollaborativeRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amazon Product Recommender Systems
## Author: Jin Hui Xu

##Collaborative Filtering 

In [1]:
!wget https://github.com/JinHuiXu1991/Jin_DATA606/blob/main/cleaned_data/cleaned_amazon_product.zip?raw=true

!wget https://github.com/JinHuiXu1991/Jin_DATA606/blob/main/cleaned_data/cleaned_amazon_review.zip?raw=true

--2022-04-04 20:55:04--  https://github.com/JinHuiXu1991/Jin_DATA606/blob/main/cleaned_data/cleaned_amazon_product.zip?raw=true
Resolving github.com (github.com)... 52.69.186.44
Connecting to github.com (github.com)|52.69.186.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/JinHuiXu1991/Jin_DATA606/raw/main/cleaned_data/cleaned_amazon_product.zip [following]
--2022-04-04 20:55:04--  https://github.com/JinHuiXu1991/Jin_DATA606/raw/main/cleaned_data/cleaned_amazon_product.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/JinHuiXu1991/Jin_DATA606/main/cleaned_data/cleaned_amazon_product.zip [following]
--2022-04-04 20:55:04--  https://raw.githubusercontent.com/JinHuiXu1991/Jin_DATA606/main/cleaned_data/cleaned_amazon_product.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133,

In [2]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 4.2 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1630165 sha256=29a6c1c8946c6dc2a7f3c13b545d4809b9e27e07b374fcd8ee427c2d8ee8e3b0
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [16]:
import pandas as pd
import numpy as np
from surprise import KNNBasic
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from sklearn.model_selection import train_test_split

In [8]:
review_df = pd.read_csv('/content/cleaned_amazon_review.zip?raw=true', compression='zip')

review_df.head()

,overall,reviewTime,reviewerID,asin,reviewText,summary,reviewTimeYear,reviewTimeMonth
0,5.0,2013-11-27,A3NHUQ33CFH3VM,1118461304,not one thing in this book seemed an obvious o...,clear on what leads to innovation,2013,11
1,5.0,2013-11-01,A3SK6VNBQDNBJE,1118461304,i have enjoyed dr alan gregerman s weekly blog...,becoming more innovative by opening yourself t...,2013,11
2,5.0,2013-10-10,A3SOFHUR27FO3K,1118461304,alan gregerman believes that innovation comes ...,the world from different perspectives,2013,10
3,5.0,2013-10-09,A1HOG1PYCAE157,1118461304,alan gregerman is a smart funny entertaining w...,strangers are your new best friends,2013,10
4,5.0,2013-09-07,A26JGAM6GZMM4V,1118461304,as i began to read this book i was again remin...,how and why it is imperative to engage learn f...,2013,9


In [10]:
#Remove all information except Reviewer ID, Product ID and rating
review_df = review_df[['reviewerID', 'asin', 'overall']]

In [4]:
product_df = pd.read_csv('/content/cleaned_amazon_product.zip?raw=true', compression='zip')

product_df.head()

,category,description,title,brand,feature,main_cat,date,price,asin,imageURLHighRes,dateYear,dateMonth,ori_title
0,"Appliances Refrigerators, Freezers & Ice Makers",NaN,tupperware freezer square round container set ...,tupperware,each 3 pc set includes two 7 8 cup 200 ml and ...,appliances,2008-11-19,NaN,7301113188,[],2008,11,Tupperware Freezer Square Round Container Set ...
1,"Appliances Refrigerators, Freezers & Ice Makers",2 x tupperware pure fresh unique covered cool ...,2 x tupperware pure amp fresh unique covered c...,tupperware,2 x tupperware pure fresh unique covered cool ...,appliances,2016-06-05,3.62,7861850250,['https://images-na.ssl-images-amazon.com/imag...,2016,6,2 X Tupperware Pure &amp; Fresh Unique Covered...
2,Appliances Parts & Accessories,NaN,the cigar moments of pleasure,the cigar book,NaN,amazon home,NaN,150.26,8792559360,['https://images-na.ssl-images-amazon.com/imag...,0,0,The Cigar - Moments of Pleasure
3,Appliances Parts & Accessories,multi purpost descaler especially suited to wa...,caraselle 2x 50g appliance descalene,caraselle,NaN,tools home improvement,2014-12-17,NaN,9792954481,['https://images-na.ssl-images-amazon.com/imag...,2014,12,Caraselle 2X 50G Appliance Descalene
4,Appliances Parts & Accessories Range Parts & A...,full gauge and size beveled edge furnished wit...,eaton wiring 39ch sp l arrow hart 1 gang chrom...,eaton wiring,returns will not be honored on this closeout i...,tools home improvement,2007-01-16,3.43,B00002N5EL,[],2007,1,EATON Wiring 39CH-SP-L Arrow Hart 1-Gang Chrom...


In [6]:
#Remove all information except Product ID and title
product_df = product_df[['asin', 'title']]

In [14]:
merged_df = review_df.merge(product_df, on='asin', how='inner')
merged_df.head()

,reviewerID,asin,overall,title
0,A3SHVDMM83IHJ4,B00002N7IL,5.0,leviton 5206 50 amp 125 250 volt nema 10 50r 3...
1,A3TIWHNJXMSIU7,B00002N7IL,5.0,leviton 5206 50 amp 125 250 volt nema 10 50r 3...
2,AVP16JFIT6LPL,B00002N7IL,5.0,leviton 5206 50 amp 125 250 volt nema 10 50r 3...
3,A2J2T9LX2HC947,B00002N7IL,5.0,leviton 5206 50 amp 125 250 volt nema 10 50r 3...
4,AJN26F4KMWD59,B00002N7IL,5.0,leviton 5206 50 amp 125 250 volt nema 10 50r 3...


In [21]:
merged_df = merged_df.groupby("reviewerID").filter(lambda x: len(x) > 1)

In [22]:
merged_df.shape

(150090, 4)

In [31]:
#Assign X as the original ratings dataframe and y as the user_id column of ratings.
X = merged_df.copy()
y = merged_df['reviewerID']

#Split into training and test datasets, stratified along user_id
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5, stratify=y, random_state=42)